In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

# Get a handle to workspace
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()
    
ml_client = MLClient.from_config(credential=credential)

In [ ]:
import os
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# Run Job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data ${{inputs.diabetes_data}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:training-data-uri-file:1"
        ),
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="ngothilinhau129152",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
)

returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

# SAU KHI SUBMIT, VÀO OUTPUT + LOGS TA THẤY CÁC FILE SAU TRONG FOLDER "model"
# conda.yaml
# MLmodel
# model.pkl
# python_env.yaml
# requirements.txt

In [ ]:
# Register the model
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

job_name = returned_job.name

run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name="mlflow-diabetes",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model)